Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

With l2 regularizer Test accuracy: 90.4%

In [6]:
batch_size = 128
hidden_size = 1024

beta = 0.01

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) # dim 128x784
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # dim 128x10
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # Connecting Input with Hidden layer
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size])) # dim 784x1024
    biases_1 = tf.Variable(tf.zeros([hidden_size])) # dim 1x1024
    # Connecting Hidden with Output layer
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels])) # dim 1024x10
    biases_2 = tf.Variable(tf.zeros([num_labels])) # dim 1x10

    # Training computation.
    # First Linear
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 # X*W1+b1 dim: 128x784 * 784x1024 + 1x1024, 128x1024+1x1024
    # ReLU Application
    relus = tf.nn.relu(logits_1) # dim 128x1024+1x1024
    # Second Linear
    logits_2 = tf.matmul(relus, weights_2) + biases_2 # ReLU(X*W1+b1)*W2+b2 dim:128x1024+1x1024 * 1024x10 + 1x10=128x10+1x10
    # Output
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
    
    # Regularization
    regularizer = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta*regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),
        weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),
        weights_2) + biases_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3492.013672
Minibatch accuracy: 7.8%
Validation accuracy: 29.8%
Minibatch loss at step 500: 21.150202
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.942810
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.603270
Minibatch accuracy: 86.7%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.614273
Minibatch accuracy: 90.6%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.645435
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.829528
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Test accuracy: 90.4%


Adam optimizer

1e-4 Test accuracy: 83.9%
1e-3 Test accuracy: 89.7%
4e-3 Test accuracy: 90.8%
5e-3 Test accuracy: 90.3%
6e-3 Test accuracy: 90.1%
1e-2 Test accuracy: 88.8%

In [12]:
batch_size = 128
hidden_size = 1024

beta = 0.01

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) # dim 128x784
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # dim 128x10
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # Connecting Input with Hidden layer
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size])) # dim 784x1024
    biases_1 = tf.Variable(tf.zeros([hidden_size])) # dim 1x1024
    # Connecting Hidden with Output layer
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels])) # dim 1024x10
    biases_2 = tf.Variable(tf.zeros([num_labels])) # dim 1x10

    # Training computation.
    # First Linear
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 # X*W1+b1 dim: 128x784 * 784x1024 + 1x1024, 128x1024+1x1024
    # ReLU Application
    relus = tf.nn.relu(logits_1) # dim 128x1024+1x1024
    # Second Linear
    logits_2 = tf.matmul(relus, weights_2) + biases_2 # ReLU(X*W1+b1)*W2+b2 dim:128x1024+1x1024 * 1024x10 + 1x10=128x10+1x10
    # Output
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
    
    # Regularization
    regularizer = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta*regularizer)
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(4e-3).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),
        weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),
        weights_2) + biases_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3499.612549
Minibatch accuracy: 14.1%
Validation accuracy: 16.5%
Minibatch loss at step 500: 493.056854
Minibatch accuracy: 79.7%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 60.035408
Minibatch accuracy: 72.7%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 11.421959
Minibatch accuracy: 85.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 4.062428
Minibatch accuracy: 90.6%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 1.848706
Minibatch accuracy: 85.2%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 1.181958
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Test accuracy: 90.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

beta = 0.01
prob = 0.5
Adam_const = 4e-3
Test accuracy: 90.6%

In [13]:
batch_size = 128
hidden_size = 1024


beta = 0.01
prob = 0.5
Adam_const = 4e-3

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size)) # dim 128x784
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # dim 128x10
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # Connecting Input with Hidden layer
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size])) # dim 784x1024
    biases_1 = tf.Variable(tf.zeros([hidden_size])) # dim 1x1024
    # Connecting Hidden with Output layer
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels])) # dim 1024x10
    biases_2 = tf.Variable(tf.zeros([num_labels])) # dim 1x10

    # Training computation.
    # First Linear
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 # X*W1+b1 dim: 128x784 * 784x1024 + 1x1024, 128x1024+1x1024
    # ReLU Application
    relu_layer = tf.nn.relu(logits_1) # dim 128x1024+1x1024

    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    # Second Linear
    logits_2 = tf.matmul(relu_layer_dropout, weights_2) + biases_2 # ReLU(X*W1+b1)*W2+b2 dim:128x1024+1x1024 * 1024x10 + 1x10=128x10+1x10
    # Output
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
    
    # Regularization
    regularizer = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta*regularizer)
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(Adam_const).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),
        weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),
        weights_2) + biases_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : prob}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))   

Initialized
Minibatch loss at step 0: 3581.815674
Minibatch accuracy: 11.7%
Validation accuracy: 18.0%
Minibatch loss at step 500: 758.515381
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 123.895348
Minibatch accuracy: 73.4%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 21.224731
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 6.361278
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 2.760426
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 1.589235
Minibatch accuracy: 79.7%
Validation accuracy: 83.6%
Test accuracy: 90.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Opis: 

2 hidden layers NN 

ReLUs
const number of nodes 1024

L2 Regularization

    (Learning rate (beta) with exponential decay) <--- do zrobienia

Dropout <--- do zrobienia

3,000 steps

==============
Test accuracy: 84.3%

In [16]:
batch_size = 128
hidden_size = 1024

beta = 0.01
prob = 0.5
Adam_const = 4e-3

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # Connecting Input with Hidden layer
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]), name='W1')
    biases_1 = tf.Variable(tf.zeros([hidden_size]), name='b1')
    # Connecting Hidden layer 1 with Hidden layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size]), name='W2')
    biases_2 = tf.Variable(tf.zeros([hidden_size]), name='b2')
    # Connecting Hidden with Output layer
    weights_3 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]), name='W3')
    biases_3 = tf.Variable(tf.zeros([num_labels]), name='b3')

    # Training computation.
    # First Linear
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    # First ReLU Application
    hidden_layer_1 = tf.nn.relu(logits_1) # dim 128x1024+1x1024
    # Second Linear
    logits_2 = tf.matmul(hidden_layer_1, weights_2) + biases_2
    # Second ReLU Application
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Third Linear
    logits_3 = tf.matmul(hidden_layer_2, weights_3) + biases_3
    # Output
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_3))
    # Regularizer
    regularizer = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3)
    # new loss
    # loss = tf.reduce_mean(loss + beta*regularizer)
    loss = loss + beta*regularizer
        
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(Adam_const).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_3)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
            tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)
            +biases_2),weights_3)
        +biases_3)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
            tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1)+biases_1),weights_2)
            +biases_2),weights_3)
        +biases_3)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14576.287109
Minibatch accuracy: 10.9%
Validation accuracy: 17.8%
Minibatch loss at step 500: 6104.221680
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 4652.479980
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 3310.090576
Minibatch accuracy: 86.7%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 2185.668457
Minibatch accuracy: 83.6%
Validation accuracy: 73.3%
Minibatch loss at step 2500: 1310.042725
Minibatch accuracy: 75.8%
Validation accuracy: 74.0%
Minibatch loss at step 3000: 702.454529
Minibatch accuracy: 78.1%
Validation accuracy: 76.5%
Test accuracy: 84.3%
